In [1]:
import pandas as pd
import numpy as np
import os
import csv
import gzip
import sys
import datetime
import ast
import matplotlib.pyplot as plt
from collections import Counter
import geopandas as gpd

In [4]:
zcta_modzcta = pd.read_csv('../data/ZCTA-to-MODZCTA.csv') ##nyc health data
zcta_modzcta.head()## int
zcta_modzcta['ZCTA_str'] = zcta_modzcta.ZCTA.astype(str)
print(zcta_modzcta['ZCTA'].nunique())
print(zcta_modzcta['MODZCTA'].nunique())

215
178


In [25]:
df_zip_zcta = pd.read_csv('../data/zip_zcta_xref.csv') ### this is acurate actully
#https://github.com/censusreporter/acs-aggregate/tree/master
modzcta_zip = zcta_modzcta.merge(df_zip_zcta, left_on='ZCTA', right_on='zcta', how='left')
zipcode_counts = pd.read_csv('../results/zipcode_counts.csv')
############# check the number of zipcodes in the survey data
### there are some zipcodes are not considered by the nyc government ###
# survey_zipcode_ls = zipcode_counts[zipcode_counts['Count']>10]['zip_code'].values.tolist()
# df_selected = modzcta_zip[modzcta_zip['zip_code'].isin(survey_zipcode_ls)]
# check_ls = df_selected['MODZCTA'].values.tolist()
# len(modzcta_zip[modzcta_zip['MODZCTA'].isin(check_ls)]) ### not equal to the df selected 

In [40]:
nyc_zip = pd.read_csv('../data/nyc_zipcodes.csv')
nyc_zip_ls = nyc_zip['nyc_zip'].values.tolist()

In [42]:
modzcta_zip_counts = pd.merge(modzcta_zip, zipcode_counts, left_on='zip_code', right_on='zip_code', how='outer')
modzcta_zip_counts = modzcta_zip_counts[modzcta_zip_counts['zip_code'].isin(nyc_zip_ls)]

In [43]:
# modzcta_zip_counts.to_csv('../results/modzcta_zip_counts.csv', index=False)
### the zipcode are individuallt mapped to the MODZCTA

In [44]:
portions = pd.read_excel('../data/TRACT_ZIP_032020.xlsx')
### it covers the whole usa 

In [45]:
### merge the with the tract data 
df_tract_modzcta = pd.merge(modzcta_zip_counts, portions, left_on='zip_code', right_on='ZIP', how='outer')

In [47]:
nyc_tract_modzcta = df_tract_modzcta[df_tract_modzcta['zip_code'].isin(nyc_zip_ls)] ### there are some zipcodes are not in the tract data

In [48]:
nyc_tract_modzcta.head() ### there are some zipcodes are not in the tract data

,ZCTA,MODZCTA,ZCTA_str,zip_code,zcta,source,Value,Count,TRACT,ZIP,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
12924,10001.0,10001.0,10001,10001.0,10001.0,geonames,10001 Chelsea - Clinton (Manhattan),22.0,36061005600,10001,0.094639,0.130724,0.007968,0.099144
12925,10001.0,10001.0,10001,10001.0,10001.0,geonames,10001 Chelsea - Clinton (Manhattan),22.0,36061005800,10001,0.253369,0.320221,0.390519,0.292938
12926,10001.0,10001.0,10001,10001.0,10001.0,geonames,10001 Chelsea - Clinton (Manhattan),22.0,36061007100,10001,0.000000,0.411816,0.090000,0.085189
12927,10001.0,10001.0,10001,10001.0,10001.0,geonames,10001 Chelsea - Clinton (Manhattan),22.0,36061007600,10001,1.000000,0.898978,0.971209,0.945020
12928,10001.0,10001.0,10001,10001.0,10001.0,geonames,10001 Chelsea - Clinton (Manhattan),22.0,36061008400,10001,0.018452,0.057143,0.135922,0.042864


In [109]:
### one GEOID has multiple MODZCTA, we choose the one with the most RES
### one zip code has multiple MODZCTA, 
for id_tochek in GOEID_MULTIPLES[2:3]:
    subdf = portions[portions['TRACT']==id_tochek]
    max_idx = subdf['TOT_RATIO'].idxmax()
    # Retrieve the row with the maximum value in 'col1'
    max_row = subdf.loc[max_idx]
    zip_to_use = max_row['ZIP'] ### int 
    to_delete = tract_mod_zip[(tract_mod_zip['GEOID_num']==id_tochek) & (tract_mod_zip['zip_code']!=zip_to_use)]
    print(to_delete)
    print('---')

       ZCTA  MODZCTA ZCTA_str  ZCTA5        GEOID  zip_code     zcta  \
1027  10024    10024    10024  10024  36061019900     10024  10024.0   
1028  10024    10024    10024  10024  36061019900     10055  10024.0   
1029  10024    10024    10024  10024  36061019900     10060  10024.0   
1030  10024    10024    10024  10024  36061019900     10087  10024.0   
1031  10024    10024    10024  10024  36061019900     10090  10024.0   
1032  10024    10024    10024  10024  36061019900     10101  10024.0   
1033  10024    10024    10024  10024  36061019900     10102  10024.0   
1034  10024    10024    10024  10024  36061019900     10108  10024.0   
1035  10024    10024    10024  10024  36061019900     10109  10024.0   
1036  10024    10024    10024  10024  36061019900     10113  10024.0   
1037  10024    10024    10024  10024  36061019900     10114  10024.0   
1038  10024    10024    10024  10024  36061019900     10116  10024.0   
1039  10024    10024    10024  10024  36061019900     10117  100

In [73]:
df_tai = pd.read_excel('../Data/2_uhfOrder.xlsx') ### this zip is not ZCTA !
df_tai['zip'] = df_tai['Zipcodes'].str.split(',')
combined_list = []
for sublist in df_tai['zip']:
    combined_list.extend(sublist)
zip_order = pd.DataFrame({'zip_tai':combined_list}) 
zip_order['Tai_idx'] = zip_order['zip_tai'].index

In [75]:
len(zip_order)

207

In [76]:
tract_mod_zip['zip_code'].nunique()

314

In [101]:
###  in our study
### the zipcode == MODZCTA
df_zipcode = pd.read_csv('zipcode_counts_new.csv')
our_study = df_zipcode['zip_code'].values.tolist()[:22]
tract_mod_zip['zip_code_int'] = tract_mod_zip['zip_code'].astype(int)
for z in our_study:
    if tract_mod_zip[tract_mod_zip['zip_code_int']==z]['MODZCTA'].nunique()!=1:
        print('zip is not unique modzcta', z)

In [162]:
cbg_data = np.loadtxt('cbgs_list.csv')